In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read and Drop

In [2]:
votes = pd.read_csv("Voting.csv")
votes.columns

Index(['FIPS', 'stateFIPS', 'state', 'county', 'stateAbb', 'region',
       'earlyVoting', 'democrat', 'republican', 'medianIncome',
       'populationOver18', 'medianAge', 'educTillHS', 'educSomeCollege',
       'educCollegeUp', 'shareWhite', 'shareBlack', 'shareAsian',
       'shareOtherRace', 'shareHispanic', 'prcpMm', 'minTempC', 'maxTempC',
       'prcpMm1015', 'minTempC1015', 'maxTempC1015', 'demSenate', 'repSenate',
       'totalSenate', 'registered', 'numPollWorkers',
       'difficultToObtainPollWorkers', 'voted', 'votedPhysical', 'votedUOCAVA',
       'votedAbsentee', 'votedProvisional', 'votedEarlyVoteCenter',
       'votedByMail', 'votedOther', 'votedOtherExplanation', 'allowSameDay',
       'repState', 'demState', 'closeElection', 'turnout', 'turnout2',
       'haveSenate', 'closeSenate', 'closePresSenate', 'freezing',
       'shareSameDay', 'shareRep'],
      dtype='object')

In [3]:
drops = ['stateFIPS', 'educTillHS', 'educSomeCollege', 'educCollegeUp',
         'state', 'shareWhite', 'shareBlack', 'shareAsian', 'shareOtherRace', 'shareHispanic', 
         'demSenate', 'repSenate', 'totalSenate', 'haveSenate', 'closeSenate', 'closePresSenate',
         'numPollWorkers', 'difficultToObtainPollWorkers', 
         'allowSameDay', 'earlyVoting', 'closeElection', 'shareSameDay',
         'county', 'stateAbb', 'region', 'medianIncome']
votes = votes.drop(columns=drops)
votes['year'] = 2016
votes.head()

,FIPS,democrat,republican,populationOver18,medianAge,prcpMm,minTempC,maxTempC,prcpMm1015,minTempC1015,...,votedByMail,votedOther,votedOtherExplanation,repState,demState,turnout,turnout2,freezing,shareRep,year
0,1001,5936.0,18172,41196,37.8,0.00,NaN,NaN,2.529167,NaN,...,NaN,NaN,NaN,1318250,729547.0,0.585202,0.610399,NaN,0.753775,2016
1,1003,18458.0,72883,155240,42.3,0.11,NaN,NaN,8.659957,NaN,...,NaN,NaN,NaN,1318250,729547.0,0.588386,0.619872,NaN,0.797922,2016
2,1005,4871.0,5454,20879,38.7,0.00,10.766667,25.566667,7.552778,5.966667,...,NaN,NaN,NaN,1318250,729547.0,0.494516,0.505005,False,0.528232,2016
3,1007,1874.0,6738,17816,40.2,0.00,10.600000,23.300000,1.950000,2.966667,...,NaN,NaN,NaN,1318250,729547.0,0.483386,0.496913,False,0.782397,2016
4,1009,2156.0,22859,44103,40.8,0.00,12.800000,21.700000,2.130952,6.275000,...,NaN,NaN,NaN,1318250,729547.0,0.567195,0.583634,False,0.913812,2016


# Ensuring there are an adequate amount of in-person votes

In [4]:
# Replace all NAs for voting numbers to 0
votes[['voted', 'votedPhysical', 'votedUOCAVA', 'votedAbsentee', 'votedProvisional', 'votedEarlyVoteCenter', 'votedByMail', 'votedOther', 'votedOtherExplanation']] = votes[['voted', 'votedPhysical', 'votedUOCAVA', 'votedAbsentee', 'votedProvisional', 'votedEarlyVoteCenter', 'votedByMail', 'votedOther', 'votedOtherExplanation']].fillna(0)

In [5]:
# Drop records that don't have a voted or votedPhysical component (no data on in-person votes)
votes.loc[((votes['votedPhysical']==0) & (votes['voted']==0)), 'noVotedData'] = True
votes = votes[votes['noVotedData'] != True]
votes = votes.drop(columns=['noVotedData'])

## 3 Cases:

In [6]:
# Useful partial for sum of non votedPhysical
votes['sumOtherVotes'] = votes['votedUOCAVA'] +  votes['votedAbsentee'] + votes['votedProvisional'] + votes['votedEarlyVoteCenter'] + votes['votedByMail'] + votes['votedOther'] + votes['votedOtherExplanation']

In [7]:
# Case 1: voted and votedPhysical are not null
votes.loc[((votes['votedPhysical']!=0) & (votes['voted']!=0)), 'sumInPerson'] = votes['votedPhysical']
votes.loc[((votes['votedPhysical']!=0) & (votes['voted']!=0)), 'sumNotInPerson'] = votes['voted'] - votes['votedPhysical']

In [8]:
# Case 2: votedPhysical is null -> use voted
votes.loc[((votes['voted']!=0) & (votes['sumInPerson'].isna())), 'sumInPerson'] = votes['voted'] - votes['sumOtherVotes']
votes.loc[((votes['voted']!=0) & (votes['sumNotInPerson'].isna())), 'sumNotInPerson'] = votes['sumOtherVotes']

In [9]:
# Case 3: voted is null -> use votedPhysical
votes.loc[((votes['votedPhysical']!=0) & (votes['sumInPerson'].isna())), 'sumInPerson'] = votes['votedPhysical']
votes.loc[((votes['votedPhysical']!=0) & (votes['sumNotInPerson'].isna())), 'sumNotInPerson'] = votes['sumOtherVotes']

In [10]:
# Drop useless columns and keep only records
votes = votes.drop(columns=['voted', 'votedPhysical', 'votedUOCAVA', 'votedAbsentee', 'votedProvisional', 'votedEarlyVoteCenter', 'votedByMail', 'votedOther', 'votedOtherExplanation', 'sumOtherVotes'])

In [11]:
votes['totalVotes'] = votes['sumInPerson'] + votes['sumNotInPerson']
votes['inPersonComp'] = votes['sumInPerson']/votes['totalVotes']
votes = votes[votes['inPersonComp'] >= .6]

# Party Lines

In [12]:
# Calculate number of 3rd party voters
votes['sumDemRep'] =  votes['democrat'] + votes['republican']
votes = votes[votes['totalVotes'] >= votes['sumDemRep']]
votes['third'] = votes['totalVotes'] - votes['sumDemRep']
votes['total']=votes['democrat']+votes['republican']+votes['third']

In [13]:
# Calculate shares of each party per registered
votes['shareRep'] = votes['republican']/votes['registered']
votes['shareDem'] = votes['democrat']/votes['registered']
votes['shareThird'] = votes['third']/votes['registered']

In [14]:
# Calculate dem or rep state
votes['isDem'] = votes['demState'] > votes['repState']

In [15]:
# Drop unused columns
votes = votes.drop(columns=['sumInPerson', 'sumNotInPerson', 'populationOver18', 'medianAge', 'totalVotes', 'inPersonComp', 'sumDemRep', 'registered', 'repState', 'demState', 'turnout', 'turnout2'])

# 2012 Votes

In [16]:
hist = pd.read_csv("historicVoting.csv")
hist['party'] = hist['party'].fillna("other")
hist.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001.0,President,Other,other,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20191203


In [17]:
hist = hist.pivot_table(index=['FIPS', 'year'], columns='party', values='candidatevotes').reset_index()
hist = hist[hist['year']==2012]

In [18]:
hist[['green', 'other']] = hist[['green', 'other']].fillna(0)
hist['third'] = hist['green']+hist['other']
hist['total'] = hist['democrat']+hist['republican']+hist['third']

hist = hist.drop(columns=['green', 'other'])

In [19]:
hist['shareDem'] = hist['democrat']/hist['total']
hist['shareRep'] = hist['republican']/hist['total']
hist['shareThird'] = hist['third']/hist['total']

In [20]:
hist = hist.merge(votes[['FIPS', 'prcpMm1015', 'isDem', 'minTempC1015', 'maxTempC1015']], on='FIPS')
hist = hist.rename({'prcpMm1015':'prcpMm', 'minTempC1015':'minTempC', 'maxTempC1015':'maxTempC'}, axis="columns")
hist['freezing'] = hist['minTempC']<=0

In [21]:
hist = hist.merge(votes[['FIPS', 'prcpMm1015', 'minTempC1015', 'maxTempC1015']], on='FIPS')

In [22]:
votes = votes.append(hist)

# Weather Finding

In [23]:
# Calculate avg temp for that election day
votes = votes[votes['minTempC'].notna() & votes['maxTempC'].notna()]
votes['avgTemp'] = votes[['minTempC', 'maxTempC']].mean(axis=1)

# Calculate avg temp of past years
votes = votes[votes['minTempC1015'].notna() & votes['maxTempC1015'].notna()]
votes['avgTempPast'] = votes[['minTempC1015', 'maxTempC1015']].mean(axis=1)

# Calculate difference between avg temp of that election day and before
votes['tempDiff'] = votes['avgTemp'] - votes['avgTempPast']

In [24]:
# Calculate difference between precipitation of that day and before
votes['prcpMm'] = votes['prcpMm'].fillna(0)
votes['rainDiff'] = votes['prcpMm'] - votes['prcpMm1015']

In [25]:
# Buckets from definitions of rain intensity
votes.loc[votes['prcpMm'] > 0, 'currRain'] = 1
votes.loc[votes['prcpMm'] >= 2.5, 'currRain'] = 2
votes.loc[votes['prcpMm'] >= 7.6, 'currRain'] = 3
votes['currRain'] = votes['currRain'].fillna(0)

votes.loc[votes['prcpMm1015'] > 0, 'pastRain'] = 1
votes.loc[votes['prcpMm1015'] >= 2.5, 'pastRain'] = 2
votes.loc[votes['prcpMm1015'] >= 7.6, 'pastRain'] = 3
votes['pastRain'] = votes['pastRain'].fillna(0)

votes['climateRainDiff'] = votes['currRain'] - votes['pastRain']
votes['raining'] = votes['currRain'] > 0

In [26]:
votes = votes.drop(columns=['minTempC', 'maxTempC', 'minTempC1015', 'maxTempC1015'])
votes = votes.sort_values(by='FIPS')

# Exporting

In [27]:
votes.to_csv("../Analysis/votesCleaned.csv")